In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_parquet('../data/ucsd-inflows.pqt', engine='pyarrow')
data

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
0,0,acc_0,PAYCHECK,2477.02,2022-03-18,PAYCHECK
1,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-10-25,EXTERNAL_TRANSFER
2,0,acc_0,MISCELLANEOUS,6.29,2022-08-26,MISCELLANEOUS
3,0,acc_0,EXTERNAL_TRANSFER,277.00,2022-06-03,EXTERNAL_TRANSFER
4,0,acc_0,EXTERNAL_TRANSFER,100.00,2022-07-29,EXTERNAL_TRANSFER
...,...,...,...,...,...,...
513110,5941,acc_9524,EXTERNAL_TRANSFER,8.66,2023-01-21,EXTERNAL_TRANSFER
513111,5941,acc_9524,EXTERNAL_TRANSFER,267.13,2023-01-23,EXTERNAL_TRANSFER
513112,5941,acc_9524,EXTERNAL_TRANSFER,2.00,2023-01-24,EXTERNAL_TRANSFER
513113,5941,acc_9524,EXTERNAL_TRANSFER,207.16,2023-01-24,EXTERNAL_TRANSFER


In [3]:
cleaned_data = data[data['category'].isin(['PAYCHECK', 'EXTERNAL_TRANSFER', 'INVESTMENT_INCOME', 'DEPOSIT', 'PAYCHECK_PLACEHOLDER', 'INSURANCE', 'OTHER_BENEFITS', 'UNEMPLOYMENT_BENEFITS'])]
cleaned_data.replace('PAYCHECK_PLACEHOLDER', 'PAYCHECK')
transaction_category = cleaned_data.groupby(['prism_consumer_id', 'category'])

In [4]:
# total dollar amount per consumer
cleaned_data.groupby(['prism_consumer_id'])['amount'].sum().sort_values(ascending=False)

prism_consumer_id
37      8262010.27
158     4096693.97
1308    2947694.28
1094    2271179.03
4901    2212199.35
           ...    
5718          0.06
824           0.05
5214          0.03
5474          0.03
5168          0.02
Name: amount, Length: 2967, dtype: float64

In [5]:
# average transaction per consumer
cleaned_data.groupby(['prism_consumer_id'])['memo'].count().mean()

102.55173576002696

In [6]:
grouped_data = cleaned_data.groupby(['prism_consumer_id', 'category']).agg(
    total_amount=('amount','sum'), 
    total_transactions=('posted_date', 'count')).reset_index()

# average amount per category
grouped_data.groupby('category')['total_amount'].mean()

category
DEPOSIT                  25114.005882
EXTERNAL_TRANSFER        29368.694075
INSURANCE                 4701.583611
INVESTMENT_INCOME         4998.024927
OTHER_BENEFITS           26469.211870
PAYCHECK                 35145.426344
PAYCHECK_PLACEHOLDER     27319.175004
UNEMPLOYMENT_BENEFITS    12108.419474
Name: total_amount, dtype: float64

In [7]:
# average transcation amount per category
grouped_data.groupby('category')['total_transactions'].mean()

category
DEPOSIT                  23.131599
EXTERNAL_TRANSFER        56.347372
INSURANCE                 2.416667
INVESTMENT_INCOME        12.572569
OTHER_BENEFITS           16.756522
PAYCHECK                 21.560182
PAYCHECK_PLACEHOLDER     19.959449
UNEMPLOYMENT_BENEFITS    14.744361
Name: total_transactions, dtype: float64

In [8]:
result = cleaned_data.groupby('prism_consumer_id').agg(
    min_date=('posted_date', 'min'),
    max_date=('posted_date', 'max'),
    total_transactions=('posted_date', 'count'),
    total_amount=('amount', 'sum')
).reset_index()

In [9]:
result['min_date'] = pd.to_datetime(result['min_date'])
result['max_date'] = pd.to_datetime(result['max_date'])
result['total_acc_months'] = ((result['max_date'] - result['min_date']).dt.days + 1) / 365

In [10]:
result

,prism_consumer_id,min_date,max_date,total_transactions,total_amount,total_acc_months
0,0,2022-02-14,2023-02-03,83,105654.93,0.972603
1,2,2021-01-13,2021-12-02,29,226065.54,0.887671
2,4,2022-02-04,2023-01-30,90,329884.21,0.989041
3,7,2022-05-06,2023-04-20,63,155608.36,0.958904
4,9,2018-10-12,2021-07-14,207,178844.31,2.758904
...,...,...,...,...,...,...
2962,5930,2021-02-09,2023-02-02,76,52280.42,1.983562
2963,5935,2021-02-17,2022-08-03,50,9212.70,1.460274
2964,5939,2021-02-01,2023-01-27,79,29962.81,1.989041
2965,5940,2021-10-08,2023-02-01,371,60191.25,1.320548


In [11]:
avg_trans = result.copy()
avg_trans = avg_trans[['prism_consumer_id', 'total_amount', 'total_transactions', 'total_acc_months']]
avg_trans.loc[:, 'trans_per_month'] = avg_trans['total_transactions'] / avg_trans['total_acc_months']
avg_trans.loc[:, 'amount_per_month'] = avg_trans['total_amount'] / avg_trans['total_acc_months']
avg_trans

,prism_consumer_id,total_amount,total_transactions,total_acc_months,trans_per_month,amount_per_month
0,0,105654.93,83,0.972603,85.338028,108631.125211
1,2,226065.54,29,0.887671,32.669753,254672.599074
2,4,329884.21,90,0.989041,90.997230,333539.436704
3,7,155608.36,63,0.958904,65.700000,162277.289714
4,9,178844.31,207,2.758904,75.029791,64824.402334
...,...,...,...,...,...,...
2962,5930,52280.42,76,1.983562,38.314917,26356.841575
2963,5935,9212.70,50,1.460274,34.240150,6308.884615
2964,5939,29962.81,79,1.989041,39.717631,15063.947176
2965,5940,60191.25,371,1.320548,280.943983,45580.510892


In [12]:
# total number of transactions per consumer per month
# number of months is the amount the consumer has an acc
avg_trans['trans_per_month'].describe()

count    2967.000000
mean      135.380638
std       132.735206
min         3.330292
25%        60.117647
50%        96.617647
75%       162.456647
max      1724.171271
Name: trans_per_month, dtype: float64

In [13]:
# average number of transactions per consumer
avg_trans['trans_per_month'].mean()

135.38063827911233

In [14]:
# total dollar amount per consumer per month
# number of months is the amount the consumer has an acc
avg_trans['amount_per_month'].describe()

count    2.967000e+03
mean     9.650955e+04
std      2.317278e+05
min      5.480480e-02
25%      2.879515e+04
50%      5.311629e+04
75%      1.005375e+05
max      8.284708e+06
Name: amount_per_month, dtype: float64

In [15]:
# average amount per consumer
avg_trans['amount_per_month'].mean()

96509.55225846882